In [5]:
import requests
import time
from bs4 import BeautifulSoup
from IPython.display import HTML, display

In [6]:
def article_info_extraction(url):
    req = requests.get(url) 
    soup = BeautifulSoup(req.text, 'lxml')
    author = soup.find('a', attrs = {'class':'author-name font-open-s'})
    if author is None:
        author = 'Noname'
    else:
        author = author.string
    title = soup.find('h1')
    title = title.string.strip()
    file = open(title + '.txt', 'w', encoding='utf-8')
    date = soup.find('p', attrs = {'class': 'date font-open-s-light'})
    file.write('@au ' + author + '\n' + '@ti ' + title
               + '\n' + '@da ' + date.string + '\n' + '@url ' + str(url) + '\n')
    description = soup.find('div', attrs = {'class': 'description font-open-s-light nm-b'})
    paragraph = description.findAll('p', attrs={'class': None})
    if len(paragraph) == 0:
        text = description.get_text()
        text = text.split('\n\n\n\n\n\n\n\n\n')
        file.write(text[0].strip())
    else:
        for part in paragraph:
            file.write(str(part.string))
    file.close()
    return 'Done'
    

In [11]:
def rednorth_crawler(num):
    pages = []
    news = []
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    url = 'https://ks-yanao.ru/novosti/'
    for i in range(1,201):
        req = requests.get(url + '?PAGEN_2=' + str(i), headers = headers)
        time.sleep(5)
        pages.append(req)
    for page in pages:
        soup = BeautifulSoup(page.text, 'lxml')
        news_per_page = soup.findAll('div', attrs = {'class':'content-body'})
        news.append(str(news_per_page))
    news = ' '.join(news[:num])
    soup = BeautifulSoup(news, 'lxml')
    for i, h in enumerate(soup.findAll('a', attrs = {'class':'news-link text-none'})):
        if h:
            h['href'] = 'https://ks-yanao.ru' + h['href'] 
            article_info_extraction(h['href'])
    return 'Done'

In [12]:
rednorth_crawler(1000)
#get_pages('https://ks-yanao.ru/novosti/')
#article_info_extraction('https://ks-yanao.ru/novosti/formula-khoroshikh-del-snova-sobiraet-poleznye-idei.html', '1')

ConnectionError: ('Connection aborted.', OSError("(54, 'ECONNRESET')",))